<style>
    .renbox {
        width: 100%;
        height: 100%;
        padding: 2px;
        margin: 3px;
        border-radius: 3px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        font-weight: bolder;
        background-color: #8b8c8b;
    }
    .per {
        background-color: #ffb654;
    }
    .org {
        background-color: #49ace6;
    }
    .misc {
        background-color: #c2ccd1;
    }
    .loc {
        background-color: #3deb6c;
    }
</style>

# Partie 1 : Entraîner un modèle de reconnaissance d'entités nommées avec SpaCy

Bonjour 👋 !

Bienvenue dans la première partie de la séquence dédiée à l'expérimentation du **traitement automatique du langage naturel** (TALN) grâce à un **modèle d'apprentissage profond**.


## Objectifs de la séance 🎯
- découvrir la reconnaissance d'entités nommées (REN)
- utiliser et entraîner un modèle REN avec la bibliothèque Python `SpaCy`
- découvrir le liage d'entités et l'expérimenter avec DBPedia
- apprendre à requêter une API Web avec la bibliothèque Python `requests`
- découvrir SPARQL, le langage d'interrogation du Web de données
- construire une carte dynamique avec la bibliothèque Python `folium`

## Important ❗

1. Répondez aux questions directement dans les cellules de ce notebook.

2. 🆘 Une question n'est pas claire ? Vous êtes bloqué(e) ?  N'attendez pas, **appelez à l'aide 🙋**.  

3. 🤖 Vous pouvez utiliser ChatGPT/Gemini/etc. pour vous aider, **mais** contraignez vous à n'utiliser ses propositions **que si vous les comprenez vraiment**. Ne devenez pas esclave de la machine ! 🙏

4. 😌 Si vous n'avez pas réussi ou pas eu le temps de répondre à une question, **pas de panique**, le répertoire `correction/` contient une solution !

ℹ️ **Info** : La difficulté d'une question **🧩**  est indiquée de ⭐ à ⭐⭐⭐⭐.

# A/ Introduction à la reconnaissance d'entités nommées

## Présentation exemplifiée 🔬
La reconnaissance d'entités nommées (REN) (en anglais *Named Entity Recognition*, ou NER) est une technique clé du traitement automatique du langage naturel (TALN). 
Elle consiste à **(1) identifier** et **(2) classer** automatiquement les **mentions d'entités du monde (réel ou fictif) dans un texte**.

Prenons l'extrait suivant : 

> « Le capitaine Smith qui commandait le Titanic est, nous l'avons dit hier, depuis plus de 35 ans au service de la White Star Line.
> Il est actuellement agé de 60 ans.
>  Né dans le Staffordshire, le capitaine Smith avait fait son apprentissage de marin dans la maison d'armement Gibson et C°, de Liverpool. »
> 
>  - dans : [Le Petit Journall, 17 avril 1912](https://gallica.bnf.fr/ark:/12148/bpt6k6196931)


La première étape est l'**identification** des mentions d'entités à l'intérieur du texte.
Cela consiste à **annoter** dans le texte les mots qui sont des mentions d'entités. 
Ici, l'extrait contient **7 mentions** de **6 entités** :

> « Le capitaine <span class ="renbox">Smith</span> qui commandait le <span class="renbox">Titanic</span> est, nous l'avons dit hier, depuis plus de 35 ans au service de la <span class ="renbox">White Star Line</span>.
> Il est actuellement agé de 60 ans.
>  Né dans le <span class ="renbox">Staffordshire</span>, le capitaine <span class ="renbox">Smith</span> avait fait son apprentissage de marin dans la maison d'armement <span class ="renbox">Gibson et C°</span>, de <span class ="renbox">Liverpool</span>. »
> 
| Entité du monde réel  | Mentions | Nombre de mentions |
| :--------------- |:---------------| :---------------|
| Le capitaine Edward John Smith | phrase 1 : "Smith", phrase 3 : "Smith"| 2 | 
| Le navire transatlantique Titanic | phrase 1 : "Titanic" | 1 | 
| La compagnie maritime White Star Line | phrase 1 : "White Star Line" | 1 | 
| Le comté anglais de Staffordshire | phrase 1 : "Staffordshire" | 1 | 
| La compagnie maritime Gibson & Co | phrase 3 : "Gibson et C°" | 1 | 
| La ville anglaise de Liverpool |  phrase 3 : "Liverpool" | 1 | 

Une fois identifiées, les mentions d'entités sont **classées** à partir d'une catégorisation prédéfinie.

Construisons nous une classification à partir de l'extrait et catégorisons les mentions d'entités avec :
| N° | Mention  | Classe |
| ---------------:| :--------------- |:---------------
|1| "Smith" | <span class="renbox per">Personne</span>| 
|2| "Titanic" | <span class="renbox misc">Objet</span> | 
|3| "White Star Line" | <span class="renbox org">Organisation</span> | 
|4| "Staffordshire" | <span class="renbox loc">Lieu</span> | 
|5| "Smith" | <span class="renbox per">Personne</span> | 
|6| "Gibson et C°" | <span class="renbox org">Organisation</span> | 
|7|  "Liverpool" |  <span class="renbox loc">Lieu</span>| 

Les mentions annotées dans le texte peuvent donc être classées :

> « Le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> qui commandait le <span class="renbox misc">Titanic<sup>[OBJET]</sup></span> est, nous l'avons dit hier, depuis plus de 35 ans au service de la <span class ="renbox org">White Star Line<sup>[ORGANISATION]</sup></span>.
> Il est actuellement agé de 60 ans.
>  Né dans le <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span>, le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> avait fait son apprentissage de marin dans la maison d'armement <span class ="renbox org">Gibson et C°<sup>[ORGANISATION]</sup></span>, de <span class ="renbox loc">Liverpool<sup>[LIEU]</sup></span>. »


## Pourquoi c'est difficile 😰

La reconnaissance d'entités nommées est l'une des tâches les plus classiques du traitement automatique du langage naturel car elle est essentielle pour **extraire des connaissances** à partir de textes.

Pourtant elle est difficile à réaliser automatiquement pour trois raisons principales : 
1. **La notion d'entité nommé est assez floue est très dépendante du domaine** et de ce que l'on cherche vraiment à extraire. Par exemple on aurait très bien pu considérer que l'expression complète <span class ="renbox per">Le capitaine Smith<sup>[PERSONNE]</sup></span> était une mention d'entité, et pas juste "Smith.".
2. **Les mentions peuvent être complexes, implicites, et contextuelles**. Dans le contexte de l'extrait, "le capitaine du navire" serait une mention de Edward Smith...
3. **Les textes eux-mêmes peuvent être complexes**. Avec des documents historiques, une source de difficulté évidente est le **bruit OCR** qui peut rendre difficile l'identification des mentions d'entités. Par exemple :

> « Le capiîe Smlth qut commandait le Tîlanic est, nous l’avons dît hîer, depuls , plus de 35 ans au servlce de la Wllhe Slar Llne. Il est actuellemenl agé de 60 ans. Né dans le Slaffordshîre, le capltalne Smlth avall fal son apprentîssage de ma-rln dans la malson d'armemenl Glbson el C, de Llverpooî. . »


La reconnaissance d'entités nommées est réalisée grâge à des techniques d'apprentissage automatique depuis les années 2010. Aujourd'hui ce sont les **modèles de langage** qui donnent les meilleurs résultats, mais le problème reste ouvert.

Pour autant, ces modèles nécessitent toujours d'être **entraînés** sur un corpus pour que les résultats soient satisfaisants.

# B/ Reconnaissance d'entités nommées avec SpaCy

## Installation 📦

[SpaCy](https://spacy.io/) est une bibliothèque logicielle Python *open source* dédiée au traitement automatique du langage naturel. 
Elle a l'avantage d'être facile d'utilisation, très documentée, très utilisée, et de mettre à disposition des modèles de langage ayant de bonnes performances.

Commençons par l'installer.

In [ ]:
# Exécutez-moi ! 🚀

%pip install -q spacy # -q (quiet) pour cacher les messages d'installation peu importants

On peut ensuite l'importer dans le notebook pour l'utiliser dans toutes les cellules suivantes.

In [ ]:
# Exécutez-moi ! 🚀

import spacy

print("Version de SpaCy installée : ",spacy.__version__)

## Télécharger un modèle de langage préentraîné 💾

Installer SpaCy a téléchargé la bibiothèque Python, mais pour qu'elle puisse réaliser des tâches de TALN il lui faut avoir accès à un **modèle de langage préentraîné**.

Explosion, l'entreprise qui développe SpaCy, en publie plusieurs spécialement entraînés pour des textes en français. On en trouve la liste ici : https://spacy.io/models/fr

Les modèles `fr_core_news_X` sont particulièrement intéressants pour nous car ils ont été entraînés sur des corpus de presse (d'où leur nom), et qu'ils sont optimisés pour fonctionner correctement sans carte graphique.

Trois modèles sont disponibles, de tailles et de performances croissantes :
| Modèle | Taille (Mb)  | Classe |
| ---------------:| :--------------- |:---------------
|1| `fr_core_news_sm` | 15Mb | 
|2| `fr_core_news_md` | 43Mb | 
|3| `fr_core_news_lg` | 545Mb |

Nous allons utiliser le modèle `fr_core_news_md` qui - on l'espère - offre un compromis intéressant entre taille (=temps de téléchargement) et performances.


Pour rendre son utilisation la plus simple possible, SpaCy mets à disposition des **commandes** qui font beaucoup de travail à notre place.

Ainsi, on peut télécharger un modèle en exécutant simplement la commande suivante :

In [ ]:
# Exécutez-moi ! 🚀

! python -m spacy download fr_core_news_md

## Reconnaître des mentions d'entités avec SpaCy 🔍

En exécutant la cellule ci-dessous, SpaCy devrait vous avoir signalé :
> You can now load the package via spacy.load('fr_core_news_md')

C'est en effet avec cette instruction Python que l'on peut charger le modèle de langage en mémoire et initialiser, dans le code, une **chaîne de traitement** (*Pipeline*) SpaCy qui permettra de traiter des textes !

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 1 - ⭐</strong></div>

L'appel à `spacy.load(...)` renvoie une "*Pipeline*" qu'on pourra ensuite utiliser.
Dans la cellule suivante,  chargez le modèle `fr_core_news_md` et stockez la *pipeline* crée dans une variable nommée `nlp`, puis affichez la liste de composants de la *pipeline* que vous pouvez récupérer avec `nlp.pipe_names`.

In [ ]:
# Complétez-moi ! 🏗️

nlp = spacy.load('fr_core_news_md')

# Print the names of the pipeline components
print(nlp.pipe_names)

Si le composant `ner` est présent, c'est parfait, la *pipeline* est donc capable du faire de la reconnaissance d'entités nommées (NER).

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>


Pour traiter un texte, rien de plus simple : il suffit de passer ce texte à la *pipeline* :

In [ ]:
# Exécutez-moi ! 🚀

texte  = """Le capitaine Smith qui commandait le Titanic est, nous l'avons dit hier, depuis plus de 35 ans au service de la White Star Line.
Il est actuellement agé de 60 ans.
Né dans le Staffordshire, le capitaine Smith avait fait son apprentissage de marin dans la maison d'armement Gibson et C°, de Liverpool.
"""

doc = nlp(texte)

Le résultat du traitement est stocké dans la variable `doc` sous la forme d'un objet `spacy.Doc`. Il s'agit d'une structure qui contient le texte, ainsi que tout ce qui en a été extrait par les différents composants de la *pipeline*.

On peut bien sûr accéder au texte lui-même :

In [ ]:
# Exécutez-moi ! 🚀

doc.text # Afficher le texte du document

... mais surtout aux **entités nommées** reconnues par le modèle !

In [ ]:
# Exécutez-moi ! 🚀

doc.ents # Afficher les entités nommées reconnues

On peut voir qu'il y a quelques différences avec notre version "manuelle" en introduction : 
- apparement, le modèle considère la mention "capitaine Smith" plutôt que simplement "Smith"
- au lieu d'une seule mention "Gibson et C°", il en a identifié deux séparées : "Gibson", et "C°".

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 2 - ⭐</strong></div>

Et les classes des mentions reconnues alors ?
Rapellez-vous, la REN contient deux étapes : l'identification des mentions et leur classification. 

SpaCy l'a fait, mais ne vous le montre pas en affichant simplement la liste des entitées nommées.
En fait, `doc.ents` stocke les entités nommées comme des objets de type `spacy.Span` qui contiennent diverses informations sur l'entité nommée dont:
- son contenu avec `ent.text` 
- sa classe avec `ent.label_` (attention à *l'underscore*)
- sa position dans le texte avec `ent.start_char` et `ent.end_char`


Complétez la cellule suivante pour afficher la liste des entités reconnues et leur classe.

In [ ]:
# Complétez-moi ! 🏗️

# Afficher les entités nommées reconnues avec leur label
for ent in doc.ents:
    print(ent.text, ent.label_)

On peut ainsi constater que l'entité nommée "Gibson" a été mal classée par le modèle comme une personne (PER), car dans ce contexte "Gibson et C°" est une entreprise.

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

On voit que le composant NER a attribué lui-même les classes PER, MISC et LOC sans qu'on ne lui en ai jamais donné la liste.
Ces classes sont prédéfinies car le composant NER du modèle `fr_core_news_md` a été entraîné spécifiquement pour les détecter.
On peut en avoir la liste complète ainsi :

In [ ]:
# Exécutez-moi ! 🚀

nlp.pipe_labels["ner"]

On voit qu'il existe aussi une classe nommée ORG...mais que désigne t-elle ? Et MISC ?
On peut le savoir grâce à `spacy.explain(LABEL)` où LABEL est une des classes connues.
Par exemple :

In [ ]:
# Exécutez-moi ! 🚀

spacy.explain("ORG") # N'hésitez pas à voir la signification des autres labels !

Plutôt que de lister les entités nommées reconnues, on peut aussi les afficher directement comme des annotations dans le texte grâce au visualiseur de SpaCY nommé `displacy`.

Par exemple :

In [ ]:
# Exécutez-moi ! 🚀

from spacy import displacy #On importe la fonction displacy

displacy.render(doc, style="ent") # style="ent" indique que l'on veut afficher les entités nommées reconnues.

# B/ Créer des données d'entraînement pour un modèle SpaCy spécialisé sur la presse française ancienne OCRisée

## Les limites d'un modèle "sur étagère" 🧱

Nous avons vu dans avec le texte d'exemple que la *pipeline* SpaCy, quoique relativement performante, faisait des erreurs pourtant évidentes:
- "White Star Line" devrait être classé comme ORG et non comme MISC;
- "Gibson et C°" devrait être une seule mention de classe ORG.

Et à votre avis, que se passe t-il avec du texte bruité par un OCR ? Exécutez la cellule suivante pour le savoir.

In [ ]:
# Exécutez-moi ! 🚀
 
texte_ocr = """Le capiîe Smlth qut commandait le Tîlanic est, nous l’avons dît hîer, depuls , plus de B5 ans au servîce de la Whîte Star Lîne.
Il est actueîlement âgé de 60 ans.
Né dans le Staffordshire, le capiîe Smitn avatt fat son apprentîssage de marîn dans la maîson d'armement Gtbson et C°, de Lîverpool."""

# Il tente de déchiffrer un texte MÉCONNAISSABLE après OCR... Le résultat va vous CHOQUER ! 🔥😱
displacy.render( nlp(texte_ocr), style="ent")

Pourquoi est-ce aussi mauvais ? Tout simplement car les modèles de SpaCy n'ont pas été entraînés sur des textes dégradés par l'OCR et sont donc incapables de comprendre la sémantique de tels textes.

Pour surmonter ce problème, une seule solution : **entraîner notre propre modèle** !

## Des données, toujours des données 🗃️

Qui dit entraînement dit apprentissage, et qui dit apprentissage dit exemples. En l'occurence, nous avons besoin de textes de presse anciens annotés d'entités nommées.

Le nerfs de la guerre de l'apprentissage automatique, ce sont les données d'entraînement : elles sont rares, coûteuses à produire (à la main ou - aujourdhui - assisté par un LLM)...

Mais comme nous sommes chanceux, la BnF a publié en 2015 dans le cadre du projet Europeana Newspaper (!) 212 extraits de presse ancienne annotés : https://api.bnf.fr/fr/texte-de-presse-annote-en-entites-nommees

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 3 - ⭐</strong></div>

Rendez-vous sur https://api.bnf.fr/fr/texte-de-presse-annote-en-entites-nommees et :
1. En suivant le lien "Textes annotés et modèle (Europeana Newspapers, 50 Mo)", téléchargez l'archive **EN-REN.zip**
2. Décompressez son contenus dans le dossier `data/` déjà créé à coté de ce notebook.
3. Le Résultat doit être un dossier `partie_1/data/Training Data Set` contenant 212 fichiers `.tag`  

Vérifiez que les fichiers sont au bon endroit en exécutant la cellule suivante.

In [ ]:
# Exécutez-moi ! 🚀

! ls -C "../data/Training Data Set" | grep .tag # Doit afficher la liste de fichiers .tag dans le dossier ./data/Training Data Set

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

Regarder le contenu d'un des ces fichiers .tag, qui sont simplement des fichiers textes :

In [ ]:
# Exécutez-moi ! 🚀

! head -n 50 "../data/Training Data Set/axaa.H.a.tag" # La commande shell head permet d'afficher les premières lignes d'un fichier (ici 50)

Que voit-on ?
1. Le texte est découpé en mots, et chaque mot est placé sur une ligne
2. Chaque ligne contient deux informations : le mot, et une étiquette
3. Les étiquettes visibles dans les 50 premières lignes sont **O**, **I-PERS** et **I-LIEU**

Ce format, pensé pour la reconnaissance d'entités nommées, se nomme **IO**.
Prenons quelques minutes pour le comprendre.

## Le format Inside-Outside (IO) pour les textes annotés d'entités nommées 🗟
<style>
    .renbox {
        width: 100%;
        height: 100%;
        padding: 2px;
        margin: 3px;
        border-radius: 3px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        font-weight: bolder;
        background-color: #8b8c8b;
    }
    .per {
        background-color: #ffb654;
    }
    .org {
        background-color: #49ace6;
    }
    .misc {
        background-color: #c2ccd1;
    }
    .loc {
        background-color: #3deb6c;
    }
</style>


Inside-Outside permet de stocker facilement des entités nommées annotées dans un texte préalablement découpé en mots (les caractères de ponctuation sont considérés comme des mots également).

Le principe est le suivant :
- tout mot qui fait partir d'une entité nommée de classe "CLASSE" est étiquetée par "I-CLASSE" - on dit qu'elle est **à l'intérieur** (*inside*, "I-") de l'entité nommée ;
- tous les autres mots sont étiquetés par "O", ils sont **en dehors** (*outside*) des entités nommées du texte.

Une manière intuitive de comprendre ce formatage est d'imaginer qu'on surligne les entités nommées dans le texte avec des surligneurs de couleurs différentes, chaque couleur correspondant à une classe.
Les mots surlignés sont notés "I", et ceux non surlignés "O".

L'extrait du fichier `axaa.H.a.tag` ci-dessus correspond donc au texte annoté :

> 
> ==>
> /Volumes/Num_TRAV$/Europeana Newspapers/NER_corpus_validation-oct2014/Extraction_pour_annotation/EXTRACTION_2/0641047/txt/X0000001.txt
> <==
> 
> e <span class ="renbox per">Emmanuel DESOLES<sup>PERS</sup></span> de LOU Directeur politique BÊ>ÀCTION ET ADMINISTRATION 9& , e <span class ="renbox loc">Rue du Pré-Botté<sup>LIEU</sup></span>, aS e <span class ="renbox loc">RENNES<sup>LIEU</sup></span> ABONNEMENTS Dép

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Il existe d'autres formats, avec leurs avantages et inconvénients. Nous n'irons pas plus loin ici, mais cet article du site Medium en fait une présentation efficace : https://medium.com/@muskaan.maurya06/name-entity-recognition-and-various-tagging-schemes-533f2ac99f52


<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span> Avez-vous remarqué que les classes du fichier annoté ne correspondent pas à celles du modèle SpaCy  (LOC, PER, ORG, MISC) ? On voit PERS et non PER pour étiqueter les mentions de personnes, LIEU et non LOC pour les lieux.
Est-ce un problème ? Oui ! Le modèle n'a aucune idée de la sémantique réelle derrière les classes, il n'aura aucun moyen de savoir que PERS et PER sont sensés désigner la même catégorie d'entités ! 
Nous aurons plus loin besoin d'harmoniser les classes pour Spacy avant des les utiliser pour entraîner un modèle. !

En découle un fait fondamental : Il est bien sûr possible de déclarer ses propres classes, inconnue du modèle, mais elles devront être **apprises totalement, sans connaissance préalable** et nécessiteront sans doute **plus d'exemples d'entraînement** pour que le modèle apprenne suffisamment bien à les reconnaître.


## Un fichier pour les assembler tous 📃

Pour se simplifier la tâche, commençons par rassembler tous les fichiers .tag en un seul grand fichier d'entraînement.

Nous pourrions le faire avec Python, mais puisque Jupyter permet d'exécuter des commandes Shell à l'aide de la *magic command* '!', autant en profiter et utiliser :

In [ ]:
# Exécutez-moi ! 🚀

! cat "../data/Training Data Set/"*.tag > "training_dataset.tag"

- La commande `cat` renvoie le contenu des fichiers en paramètre, ici tous les fichiers finissant par .tag dans le dossier ./data/Training Data Set
- '>' sert à rediriger la sortie de la commande précédente, ici le contenu de tous les fichiers .tag affichés par `cat`
- la sortie est redirigée vers un fichier nommé "training_dataset.tag"

## Harmonisation des classes d'entités nommées et séparation des phrases 🧹

Les classes d'entités nommées annotées dans les fichier .tag sont PERS, LIEU et ORG, qui correspondent respectivement aux classes de SpaCy PER, LOC et ORG.

Pour que le modèle s'y retrouve, nous devons donc retoucher le fichier training_dataset.tag et remplacer toutes les occurrences des classes initiales par les classes SpaCy.

En plus, la [documentation de SpaCy](https://spacy.io/api/cli#convert) indique que dans le format IO, les phrases doivent être séparées par des lignes vides - ce qui n'est pas le cas dans les fichiers .tag.
Nous devons donc insérer des lignes vides après chaque fin de phrase.

N'étant ni passionnantes ni l'objectif pédagogique de la séance, le code de ces opérations de pré-traitement vous est donné directement dans la cellule ci-dessous, à exécuter.

In [ ]:
# Exécutez-moi ! 🚀

# Liste de remplacement , [(ancien_label, nouveau_label), ...]
mapping = [
    ("I-LIEU", "I-LOC"),
    ("I-PERS", "I-PER"),
    ("I-ORG", "I-ORG")
]

# On ouvre le fichier en mode lecture...
with open("training_dataset.tag", "r") as file:
    training_dataset = file.read()
    # ... on remplace les labels anciens par les nouveaux ...
    for k, v in mapping:
        training_dataset = training_dataset.replace(k, v) 

    # ... et on insère des lignes vides(2 retours à la ligne) à chaque marqueur de fin de phrase, ie. une ligne ". O"
    import re
    training_dataset = re.sub(r"(\n\.\tO)", r"\1\n", training_dataset)

# Finalement, on ouvre le fichier en mode écriture et on écrit le contenu modifié
with open("training_dataset.tag", "w") as file:
    file.write(training_dataset)

On peut vérifier que l'opération semble s'être bien passée en regardant le début du fichier grâce à la commande Shell `head`.

In [ ]:
# Exécutez-moi !  🚀

# Les 50 premières lignes du fichier "training_dataset.tag"
! head -n 50 "training_dataset.tag"

## Transformation en documents SpaCy 🪄

Évidemment, SpaCy ne sait pas lire le format IO mais uniquement son format propre - ce serait trop simple.
Heureusement, la bibliothèque fournit une commande de conversion : `spacy convert` (doc : https://spacy.io/api/cli#convert )

Décomposons la commande complète dont nous avons besoin :
```bash
python -m spacy convert -c ner  -n 10 -b fr_core_news_md "training_dataset.tag" .
```
- l'option `-c ner` indique à SpaCy que le format de training_dataset.tag est IO ou similaire, en tout cas un format d'annotations de reconnaissance d'entités nommées ;
- Avec `-n 10`, Spacy va découper le jeu de données en groupes de 10 phrases, chaque groupe sera un exemple d'entraînement.
- `-b fr_core_news_md` spécifie à SpaCy qu'il doit utiliser le modèle `fr_core_news_md` pour effectuer la transformation. Il a besoin de connaître le modèle car chaque mot va être codé par un identifiant, et c'est le modèle qui fournit cela.

Appliquons la conversion pour produire un nouveau fichier `training_dataset.spacy`, au format SpaCy.

<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span> La commande ci-dessus doit produire un fichier contenant **1191 documents**. SI ce n'est pas le cas, signalez-le !

In [ ]:
# Exécutez-moi ! 🚀

! python -m spacy convert -c ner  -n 10 -b fr_core_news_md "training_dataset.tag" .

Ouf, nous avons enfin notre fichier principal d'entraînement !

En a t-on fini ? Presque mais pas tout à fait : il faut encore séparer le fichier en deux sous-ensembles d'exemples : les **exemples d'entraînement** et les **exemples d'évaluation**.

## Jeu d'entraînement, jeu d'évaluation, jeu de test 🗂️

Lors de son entraînement, un modèle va exploiter un ensemble d'exemples qu'on lui aura fourni; ces exemples forment le **jeu (de données) d'entraînement** du modèle.

Durant l'entraînement, le modèle va régulièrement mesurer sa performance en comparant ses prédictions d'entités nommées avec des exemples.
Mais **on ne veut pas que ces exemples proviennent du jeu d'entraînement**.
Pourquoi ? Parce que s'il fait cela, il apprendra **par coeur** ses exemples d'entraînement et n'apprendra pas à **généraliser**, c'est à dire à dégager une sémantique de plus haut niveau qui lui permettra de "comprendre" correctement des textes.

Pour résoudre le problème, on isole un ensemble d'exemples qui forme un **jeu d'évaluation**, sur lequel le modèle n'apprends pas directement mais qu'il utilisera pour tester ses performances en cours d'apprentissage.

Enfin, il est important de conserver une partie des données totalement hors du processus d'apprentissage pour pouvoir **tester** les performances du modèle en **situation réelle**, c'est à dire en le confrontant à des exemples qu'il n'aura jamais vu. On appelle cet ensemble le **jeu de test**.

<span style="color: #fc03d3; font-size: 1.2em;"><strong>📝 À retenir |</strong></span>
Pour résumer : 
- le modèle apprends à reconnaître des entités nommées dans les exemples du **jeu d'apprentissage** ;
- pour vérifier qu'il apprends correctement, il mesure ses performances face à des exemples différents appelés **jeu d'évaluation**.
- on garde des exemples complètement hors du circuit d'apprentissage pour tester les capacités du modèle une fois entraîné:  c'est le **jeu de test**.


En règle générale, on conserve environ 80% du jeu de données total comme jeu d'entraînement, et donc 20% comme jeu d'évaluation.
Ici, on commencera par séparer 10% du corpus générale en un jeu de test, pour appliquer la règle des 80/20% sur le reste.

Il faut donc séparer le fichier `train_dataset.spacy` en trois nouveaux fichiers : `test.spacy` qui contiendra 10% du total des documents, `train.spacy` qui contiendra 80% des documents restants, et `evaluation.spacy` qui contiendra les 20 autres %.

Nous pourrions faire cela à la main ... mais nous pouvons aussi faire appel à la fonction `train_test_split` de la bibliothèque Python `scikit-learn` qu'il nous faut installer (doc : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
# Exécutez-moi ! 🚀

%pip install -q scikit-learn

La fonction `train_test_split`  accepte deux arguments principaux : 
- une liste d'exemples à séparer
- une option `train_size=...` ou `test_size=...` qui prend un nombre entre 0 et 1 représentant la fraction d'exemples qui doivent composer le  jeu d'entraînement, ou de test (d'évaluation, pour nous).

Cette fonction renvoie un tuple contenant **deux listes** : (1) la liste des  exemples d'entraînement et (2) la liste des exemples d'évaluation.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 4 - ⭐⭐</strong></div>

Complétez puis exécutez le code suivant pour séparer les documents dans `tranining_dataset.spacy` en deux ensembles :
- `docs_test` doit contenir 10% des exemples ;
- `doc_train_evaluation` doit contenir les 90% restants

Ensuite, séparez de nouveau `doc_train_evaluation` en deux ensembles : 
 - `train.spacy` doit contenir 80% des exemples ;
 - `evaluation.spacy` doit contenir les 20% restants ;


In [ ]:
# Complétez-moi ! 🏗️

from spacy.tokens import DocBin # DocBin est une classe qui représente un jeu de données SpaCy
from sklearn.model_selection import train_test_split  # sci-kit learn s'importe sous le nom sklearn

# Chargement du jeu de données SpaCy des textes annotés
jeu_complet = DocBin().from_disk("training_dataset.spacy")

# Chargement du modèle de langue SpaCy pour le français
nlp = spacy.load('fr_core_news_md')

# Récupération des documents du jeu de données sous forme de liste
docs = list(jeu_complet.get_docs(nlp.vocab))

# Complétez ici en appellant la fonction train_test_split de scikit-learn !
# Étape 1 : séparation en deux listes : `docs_test` (10%) et `docs_train_evaluation` (90%)
# Étape 2 : séparation de `docs_train_evaluation` en deux listes : `docs_train` (80%) et `docs_evaluation` (20%)
docs_test, docs_train_evaluation = train_test_split(docs, train_size=0.1)
docs_train, docs_evaluation = train_test_split(docs_train_evaluation, train_size=0.8)

# Export des jeux de données d'entraînement et de test
DocBin(docs=docs_train).to_disk("train.spacy")
DocBin(docs=docs_evaluation).to_disk("evaluation.spacy")
DocBin(docs=docs_test).to_disk("test.spacy")

# Afficher la taille des jeux de données d'entraînement et d'évaluation
print("Taille du jeu d'entraînement : ", len(docs_train))
print("Taille du jeu d'évaluation : ", len(docs_evaluation))
print("Taille du jeu de test : ", len(docs_test))

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

Vérifions enfin que le répertoire courant contient bien les deux fichiers utiles :

In [ ]:
# Exécutez-moi ! 🚀

! ls -lh train.spacy evaluation.spacy

Ouf ! Nous voilà prêts pour le moment de vérité : **l'entraînement du modèle !**.

# C/ Entraîner un modèle SpaCy spécialisé sur la presse française ancienne OCRisée

Contrairement aux modèles Transfromers (BERT, etc.), les modèles `core` de SpaCy (comme `fr_core_news_x`) ne sont pas adaptés au *fine-tuning*, c'est à dire à la spécialisation d'un modèle déjà entraîné : avec les modèles `core`, on entraîne *from scratch*.


## Configuration de l'entraînement ⚙️ 

Le paramètrage de l'entraînement d'un modèle SpaCy se fait à l'aide de fichiers de configuration.
Heureusement, on trouve à la page [https://spacy.io/usage/training ](https://spacy.io/usage/training), section *Quickstart*, un assistant de création de tels fichiers.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 5 - </strong></div>

Avec l'assistant, créer un fichier pour le **français**, contenant le composant **ner**, pour **CPU** et optimisé pour **l'efficience**.

Téléchargez le fichier produit (le bouton est en bas à droite du *widget*) et placez-le dans le dosser de travail `partie_1/`.

La documentation nous dit d'exécuter ensuite la commande SpaCy suivante pour produire le fichier de configuration complet à partir du fichier téléchargé :

In [ ]:
# Exécutez-moi ! 🚀

! python -m spacy init fill-config base_config.cfg config.cfg

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>


## On entraîne ! 🔥🚀

Tout est prêt, nous avons :
-  le fichier de configuration complet `config.cfg`
-  le jeu d'entraînement `train.spacy`
-  le jeu d'évaluation `evaluation.spacy`

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 6 - ⭐</strong></div>

Servez-vous du message de sortie de la commande précédente pour lancer l'entraînement.

Utilisez l'option `--output` pour **sauvegarder le modèle enbtraîné dans le dossier `ner_presse_ancienne/`**

<span style="color: #85d0ff; font-size: 1.2em;"><strong>ℹ️ Info |</strong></span> Le jeu d'évaluation est désigné dans SpaCy par le terme "dev", pour "dévelopment" - mais c'est bien notre jeu d'évaluation qu'il faut utiliser !

<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span> Si l'entraînement est trop lent, basculez sur Google Colab et **demandez de l'aide** 🙋.

In [ ]:
# Complétez-moi ! 🏗️

# Lancez l'entraînement du modèle NER !
! python -m spacy train config.cfg --paths.train train.spacy --paths.dev evaluation.spacy --output ./ner_presse_ancienne

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

Durant l'entraînement SpaCy affiche tous les K itérations un ensemble de valeurs,par exemple : 
```raw
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     64.29    0.09    0.17    0.06    0.00
  0     200        162.71   2838.80   40.70   52.57   33.20    0.41
  0     400         96.11   1467.98   49.21   65.45   39.43    0.49
```

Sans entrer dans les détails qui dépasseraient l'objectif de cet atelier, deux choses sont à retenir:
1. l'entraînement d'un modèle statistique est un problème d'**optimisation** où l'on cherche à minimiser une **fonction de coût** (LOSS) qui mesure l'écart entre les prédictions du modèle et les exemples qui lui sont donnés. C'est la valeur donnée par `LOSS NER`.
2. le jeu d'évaluation est utilisé en cours d'apprentissage pour mesurer la propension du modèle à identifier et classer les entités nommées attendues. Ce sont les mesures `ENTS_F`, `ENTS_P` et `ENTS_R`. Nous verrons plus loin ce que les suffixes `_F`, `_P` et `_R` signifient.

<span style="color: #fc03d3; font-size: 1.2em;"><strong>📝 À retenir |</strong></span>
En bref, le modèle améliore ses performances lorque :  
- LOSS LOTK2VEC et LOSS NER **diminuent**
- ENTS_F, ENTS_P  et ENTS_R **augmentent**



Regardons finalement ce qui a été exporté par Spacy

In [ ]:
# Exécutez-moi ! 🚀

! tree -L 2 ./ner_presse_ancienne

On peut voir que Spacy a en réalité exporté **deux modèles** :
- `model_best/` contient le modèle ayant montré les meilleures performances durant les itérations d'entraînement (ie. la ligne du tableau d'entraînement avec les meilleures mesures ENTS) ;
- `model_last/` contient le modèle à la fin de l'entraînement. 

<span style="color: #fc03d3; font-size: 1.2em;"><strong>📝 À retenir |</strong></span> L'entraînement d'un modèle est un processus stochastique, le modèle produit en fin de processus n'est pas nécessairement le meilleur !

## Évaluation quantitative du modèle 💯

Évaluer les performances d'un modèle sur des données **qu'il n'a jamais vu lors de son entraînement** est une étape essentielle de toute procédure d'entraînement.
Cela mesure sa capacité à **généraliser** ce qu'il a appris pour de nouveaux textes. 

Nous allons pour cela faire appel au **jeu de test** créé précédement et utiliser la commande `spacy benchmark accuracy` (https://spacy.io/api/cli#benchmark-accuracy) pour tester les performances du meilleur modèle (stocké dans `ner_presse_ancienne/model-best/`)

In [ ]:
# Exécutez-moi ! 🚀

! python -m spacy benchmark accuracy ner_presse_ancienne/model-best/ test.spacy 

Le résultat devrait ressembler à ceci  :
```raw
================================== Results ==================================
TOK     -    
NER P   69.72
NER R   64.39
NER F   66.95
SPEED   41713
=============================== NER (per type) ===============================
          P       R       F
LOC   76.02   62.14   68.38
ORG   50.38   42.58   46.15
PER   70.24   75.64   72.84
```

Comment interprêter ce rendu un peu obscur ?
Regardons d'abord la section `====Results====`, qui mesure la qualité globale de la reconnaissance des entités dans les textes du **jeu de test**.

Les lettres `P`, `R` et `F` sont des abbréviations de trois métriques de qualité très communément utilisées pour ce type de tâche :

|Abbréviation|Nom|Description|
|:------|:------|---------|
|P| Précision |	Pourcentage d'entités nommées prédites par le modèle qui étaient effectivement dans le jeu de test. Une précision élevée signifie que lorsque le modèle prédit une entité nommée, il a souvent raison. |
|R| Rappel	| Pourcentage des entités nommées du jeu de test que le modèle a effectivement trouvé. Un rappel élevé signifie que le modèle a trouvé la majorité des entités nommées du jeu de test |  
|F| F-Score|	La moyenne harmonique de la précision et du rappel, mélange les deux mesures en un unique score. |

Nous pouvons donc interprêter les résultats ainsi : 
- 69.72% des prédictions du modèle étaient correctes ;
-  il a trouvé 64.39% des entités nommées du jeu de test ;
- sa performance générale de reconnaissance est de 66.95 / 100 (sans unité)


<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 6 - ⭐</strong></div>

Sachant cela, comment interprétez-vous la section `==== NER (per type) ===` ?

Qu'est-ce qui pourrait expliquer que les résultats ne soient pas meilleurs ? Discutons-en ensemble ! 🙋

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>


<span style="color: #fc03d3; font-size: 1.2em;"><strong>📝 À retenir |</strong></span> 
- Une **précision** élevée est importante si on veut **éviter les fausses détection** ;
- Un **rappel** élevé est important si on on **éviter d'oublier des entités nommées** ;
- Un **F-score** élevé est important si on veut concilier précision et rappel !

## Évaluation qualitative du modèle 👀


Si l'évaluation quantitative d'un modèle est essentielle, observer qualitativement ses performances sur des exemples est également crucial pour mieux comprendre ses erreurs.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 7 - ⭐</strong></div>

Complétez la cellule suivante pour tester le modèle entraîné sur l'extrait OCRIsé de l'article sur le naufrage du Titanic **et afficher le texte annoter avec displacy**

Les résultats sont-ils meilleurs qu'avec le modèle `fr_core_news_md` ? Où sont les erreurs s'il en reste ? Comment les expliquer ?

In [ ]:
# Complétez-moi ! 🏗️

texte_ocr = """Le capiîe Smlth qut commandait le Tîlanic est, nous l’avons dît hîer, depuls , plus de B5 ans au servîce de la Whîte Star Lîne.
Il est actueîlement âgé de 60 ans.
Né dans le Staffordshire, le capiîe Smitn avatt fat son apprentîssage de marîn dans la maîson d'armement Gtbson et C°, de Lîverpool."""

nlp = spacy.load('ner_presse_ancienne/model-best')

displacy.render(nlp(texte_ocr), style="ent")

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

# Ouf, c'est fini ! 🏁

C'est tout pour cette fois, vous voici arrivé(e)s au bout, félicitations ! 🎉🎉

<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span> Conservez le dossier `ner_presse_ancienne/`, vous en aurez besoin pour la prochaine séance !